## Using HuggingFace Model to try Spanish Texts

#### Installing Libraries

In [1]:
pip install transformers --q

Note: you may need to restart the kernel to use updated packages.


In [2]:
pip install "transformers[sentencepiece]" --q

Note: you may need to restart the kernel to use updated packages.


In [5]:
pip install torch --q

Note: you may need to restart the kernel to use updated packages.


In [6]:
import transformers

/Users/felipe_q/anaconda3/envs/mlflow/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


##### We load the model in spanish
The english model is: 'distilbert-base-uncased-finetuned-sst-2-english'

Note: We are getting an error downloading the fine tuned version for the spanish texts model, thus we will keep developing the english example provided by Hugging Face

In [7]:
from transformers import AutoTokenizer
#checkpoint = 'distilbert-base-multilingual-cased-sentiment'
checkpoint = 'distilbert-base-uncased-finetuned-sst-2-english' 
#checkpoint = 'distilbert-base-multilingual-cased'
tokenizer = AutoTokenizer.from_pretrained(checkpoint)

In [14]:
raw_inputs = [
    'I ve been waiting for a HuggingFace course my whole life.',
    'I hate this so much!'
]

inputs = tokenizer(raw_inputs, padding = True, truncation = True, return_tensors = 'pt')
print(inputs)

{'input_ids': tensor([[  101,  1045,  2310,  2042,  3403,  2005,  1037, 17662, 12172,  2607,
          2026,  2878,  2166,  1012,   102],
        [  101,  1045,  5223,  2023,  2061,  2172,   999,   102,     0,     0,
             0,     0,     0,     0,     0]]), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1],
        [1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0]])}


In [15]:
from transformers import AutoModel

model = AutoModel.from_pretrained(checkpoint)

In [16]:
outputs = model(**inputs)
print(outputs.last_hidden_state.shape)

torch.Size([2, 15, 768])


Now we import the model head for sequence classification


In [18]:
raw_inputs = [
    'I ve been waiting for a HuggingFace course my whole life.',
    'I hate this so much!',
    'I love this'
]

inputs = tokenizer(raw_inputs, padding = True, truncation = True, return_tensors = 'pt')
print(inputs)

{'input_ids': tensor([[  101,  1045,  2310,  2042,  3403,  2005,  1037, 17662, 12172,  2607,
          2026,  2878,  2166,  1012,   102],
        [  101,  1045,  5223,  2023,  2061,  2172,   999,   102,     0,     0,
             0,     0,     0,     0,     0],
        [  101,  1045,  2293,  2023,   102,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0]]), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1],
        [1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0],
        [1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]])}


In [19]:
from transformers import AutoModelForSequenceClassification

model = AutoModelForSequenceClassification.from_pretrained(checkpoint)
outputs = model(**inputs)

print(outputs.logits.shape)

torch.Size([3, 2])


In [20]:
print(outputs.logits)

tensor([[-1.4231,  1.4773],
        [ 4.1692, -3.3464],
        [-4.3421,  4.7022]], grad_fn=<AddmmBackward0>)


In [24]:
import torch

predictions = torch.nn.functional.softmax(outputs.logits, dim = -1)
print(predictions)

tensor([[5.2132e-02, 9.4787e-01],
        [9.9946e-01, 5.4418e-04],
        [1.1804e-04, 9.9988e-01]], grad_fn=<SoftmaxBackward0>)


In [25]:
model.config.id2label

{0: 'NEGATIVE', 1: 'POSITIVE'}